In [95]:
!pip install gensim
workingdir = "C:/Users/arjain/Box/ML Class Project/"
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import *
import os, glob
import re

import datetime
from datetime import datetime
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import gensim
from gensim import corpora
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from itertools import chain
from sklearn.model_selection import train_test_split

from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases,ENGLISH_CONNECTOR_WORDS
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import TfidfModel
from gensim.parsing.preprocessing import strip_short
import matplotlib.pyplot as plt 

C:\Users\arjain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arjain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arjain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\arjain\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [150]:
def process(doc):
    #except for company names,
    #make all text lowercase, remove everything except alpha chars
    #sentences = doc.splitlines()
    stop_free = " ".join([i.lower() for i in doc.split() if i.lower() not in set(stopwords.words('english'))])

    #remove all punctuation except for tickers
    punc_free = ''
    for word in stop_free.split():
        temp = word
        if len(word)!=0:
            if word[0]!='#':
                temp = re.sub('\.', '', word)
                temp = re.sub('[^a-zA-Z#\s]', ' ', temp)
        temp = temp.strip()
        punc_free = punc_free + ' ' + temp        
    punc_free = re.sub('\ss\s', ' ', punc_free)
    punc_free = re.sub('\s\s+', ' ', punc_free)
    normalized = [lemma.lemmatize(word) for word in punc_free.split()]
    return normalized


def cross_val_k( test_data, train_data):
    chsize = 5000
    train_data_matrix = [dictionary.doc2bow(doc) for doc in train_data]
    tfmodel = TfidfModel(train_data_matrix)
    train_data_matrix = tfmodel[train_data_matrix]
    test_data_matrix = [dictionary.doc2bow(doc) for doc in test_data]
    tfmodel = TfidfModel(train_data_matrix)
    test_data_matrix = tfmodel[test_data_matrix]
    
    ks = []
    scores = []
    k=5
    while k <= 50 :
        
        ldamodel = gensim.models.LdaMulticore(corpus=train_data_matrix,
                                              id2word=dictionary,
                                              num_topics=k,
                                              chunksize=chsize)
        coherence_model_lda = CoherenceModel(model=ldamodel, texts=test_data, 
                                             dictionary=dictionary, coherence='c_v')
        
        
        ks += [k]
        scores += [coherence_model_lda.get_coherence()]

        k += 1

    plt.plot(ks, scores) 
    
    return None

articles = pd.read_csv(workingdir+"articles_cleaned.csv")
article_Week= pd.read_csv(workingdir+"articles_cleaned_1wkafter_ipo.csv")
articles_3=pd.read_csv(workingdir+"articles_cleaned_3.csv")
articles_df=pd.concat([articles, article_Week,articles_3])
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()
docs = articles_df['cleaned_text']
docs = [doc.splitlines() for doc in docs]
docs = list(chain.from_iterable(docs))
docs = [x for x in docs if len(x)>0]

doc_cleaned = [process(x) for x in docs]
doc_cleaned = [x for x in doc_cleaned if len(x)>5]


phrase_model = Phrases(doc_cleaned, min_count=1, threshold=100, connector_words=ENGLISH_CONNECTOR_WORDS)
doc_cleaned = phrase_model[doc_cleaned]


# Filter out words that occur less than 10 documents, or more than 50% of the documents.






C:\Users\arjain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:13: DeprecationWarning: invalid escape sequence \.
<>:14: DeprecationWarning: invalid escape sequence \s
<>:17: DeprecationWarning: invalid escape sequence \s
<>:18: DeprecationWarning: invalid escape sequence \s
<>:13: DeprecationWarning: invalid escape sequence \.
<>:14: DeprecationWarning: invalid escape sequence \s
<>:17: DeprecationWarning: invalid escape sequence \s
<>:18: DeprecationWarning: invalid escape sequence \s
<ipython-input-150-05377037c757>:13: DeprecationWarning: invalid escape sequence \.
  temp = re.sub('\.', '', word)
<ipython-input-150-05377037c757>:14: DeprecationWarning: invalid escape sequence \s
  temp = 

In [151]:

docu_clean=[]
for x in doc_cleaned:
    x=[i for i in x if i != "inc"]
    x=[i for i in x if len(i)>2]
    x=[i for i in x if i != "million"]
    x=[i for i in x if i != "llc"]
    x=[i for i in x if i != "co"]
    x=[i for i in x if i != "billion"]
    x=[i for i in x if i != "united"]
    x=[i for i in x if i != "find_article"]
    x=[i for i in x if i != "useful_subscribe"]
    x=[i for i in x if i != "states"]
    x=[i for i in x if i != "m_shs"]
    x=[i for i in x if i != "keywords_news"]
    x=[i for i in x if i != "contact_information"]
    x=[i for i in x if i != "canadian"]
    x=[i for i in x if i != "usd"]
    x=[i for i in x if i != "journal_article"]
    x=[i for i in x if i != "bizjournalscom_subscribe"]
    x=[i for i in x if i != "ltd"]
    x=[i for i in x if i != "copyright_newsrx"]
    x=[i for i in x if i != "sic_code"]
    x=[i for i in x if i != "the"]
    x=[i for i in x if i != "that"]
    x=[i for i in x if i != "there"]
    x=[i for i in x if i != "said"]
   
   
   
   
   
    docu_clean.append(x)


print (doc_cleaned[1])
print (docu_clean[1])
        
      







C:\Users\arjain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['#aten#', 'inc', 'announces_pricing', 'initial', 'public', 'offering']
['#aten#', 'announces_pricing', 'initial', 'public', 'offering']


train_data, test_data = train_test_split(doc_cleaned, test_size=0.2)
cross_val_k( test_data, train_data)
articles_df['sentences'] = [doc.splitlines() for doc in articles_df['cleaned_text']]
articles_df['sentences'] = [ [x for x in sentences if len(x) > 0] for sentences in articles_df['sentences']]
articles_df['sentences'] = [ [process(x) for x in sentences] for sentences in articles_df['sentences']]
articles_df['sentences'] = [ [x for x in sentences if len(x) > 2] for sentences in articles_df['sentences']]
articles_df['sentences_lda'] = [ [phrase_model[x] for x in sentences] for sentences in articles_df['sentences']]
articles_df['sentences_lda'] = [ [dictionary.doc2bow(x) for x in sentences] for sentences in articles_df['sentences_lda']]
#get most likely topic for sentence
articles_df['sentences_lda'] = [ [np.argmax([ x[1] for x in ldamodel[sen]]) for sen in sentences] for sentences in articles_df['sentences_lda']]

In [152]:
dictionary = corpora.Dictionary(docu_clean)
dictionary.filter_extremes(no_below=50, no_above=0.5)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in docu_clean]
tfmodel = TfidfModel(doc_term_matrix)
doc_term_matrix = tfmodel[doc_term_matrix]


C:\Users\arjain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [147]:
ldamodel = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                     id2word=dictionary,
                     num_topics=30,
                     chunksize=50,passes=10)
ldamodel.print_topics()

ldamodel.save('ldamodel.model')
ldamodel.print_topics()

C:\Users\arjain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(25,
  '0.118*"call" + 0.100*"may" + 0.035*"nomura" + 0.030*"view" + 0.028*"model" + 0.026*"rise" + 0.026*"plan" + 0.024*"option" + 0.022*"general" + 0.021*"working"'),
 (22,
  '0.135*"jefferies" + 0.106*"raise" + 0.094*"credit_suisse" + 0.091*"price" + 0.084*"deutsche_bank" + 0.077*"raymond_james" + 0.077*"bmo" + 0.061*"citigroup" + 0.060*"corp" + 0.017*"street"'),
 (11,
  '0.115*"report" + 0.112*"deliver_fact" + 0.112*"research_discovery" + 0.110*"around_world" + 0.108*"news" + 0.067*"based" + 0.051*"system" + 0.031*"demand" + 0.026*"annual" + 0.023*"order"'),
 (10,
  '0.080*"company" + 0.039*"wall_street" + 0.035*"several" + 0.035*"#wday#" + 0.028*"far" + 0.027*"recent" + 0.027*"update" + 0.024*"#dell#" + 0.024*"including" + 0.023*"drive"'),
 (19,
  '0.241*"corp" + 0.045*"reuters" + 0.043*"research" + 0.040*"property" + 0.020*"nov" + 0.017*"already" + 0.016*"large" + 0.015*"owner" + 0.015*"oil" + 0.015*"area"'),
 (16,
  '0.082*"security" + 0.075*"form" + 0.056*"statement" + 0.041*"

In [153]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary=ldamodel.id2word)
vis

C:\Users\arjain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
5     -0.166486 -0.034356       1        1  6.664272
13    -0.201972  0.086231       2        1  6.524880
3     -0.141799  0.110247       3        1  6.203156
6     -0.177895  0.026225       4        1  5.514314
18    -0.136831 -0.001463       5        1  4.679615
24    -0.111934 -0.023034       6        1  3.927952
26     0.254772  0.068536       7        1  3.883548
12    -0.125590  0.039672       8        1  3.374710
16    -0.031931 -0.116688       9        1  3.353169
17    -0.072822  0.040978      10        1  3.345794
22     0.266268  0.180228      11        1  3.211675
2      0.209607  0.183423      12        1  3.183470
29    -0.052577  0.021034      13        1  3.099958
14    -0.110056  0.039452      14        1  3.038690
19     0.045268  0.087816      15        1  2.934455
10    -0.112440  0.000957      16        1  2.932830
25     0.015197 -0.009471      17        1  2.844761
4      0.030340  0.011958      18        1  2.758577
0     -0.120494  0.016980      19        1  2.725198
15     0.209526  0.079506      20        1  2.692564
7      0.052539 -0.047314      21        1  2.551643
23    -0.033770  0.062929      22        1  2.534295
21     0.064613 -0.004440      23        1  2.524897
9      0.048669 -0.227935      24        1  2.493881
28    -0.018548  0.055247      25        1  2.430175
20     0.098017 -0.319461      26        1  2.312441
27     0.054706 -0.008839      27        1  2.145826
1      0.046111 -0.035758      28        1  2.076736
8      0.122058 -0.231018      29        1  2.028953
11     0.097455 -0.051642      30        1  2.007561, topic_info=           Term         Freq        Total Category  logprob  loglift
707      target  9040.000000  9040.000000  Default  30.0000  30.0000
26        price  8147.000000  8147.000000  Default  29.0000  29.0000
1357     rating  4782.000000  4782.000000  Default  28.0000  28.0000
736       raise  6957.000000  6957.000000  Default  27.0000  27.0000
953        corp  5431.000000  5431.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
143       based   686.273217  2093.814105  Topic30  -2.6979   2.7928
20        march   214.379287   662.675236  Topic30  -3.8615   2.7797
507   corporate   120.225607   315.175117  Topic30  -4.4398   2.9445
244     process    95.072305   318.433661  Topic30  -4.6746   2.6995
81    financial    38.051615  2311.837943  Topic30  -5.5903  -0.1986

[1398 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
750      26  0.990778   #acia#
1000     22  0.861922   #acmr#
179       1  0.139255   #aone#
179       2  0.124335   #aone#
179       4  0.073890   #aone#
...     ...       ...      ...
37        9  0.638477     york
37       11  0.031145     york
37       13  0.019466     york
37       22  0.009084     york
267      18  0.985325  york_ap

[3112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 14, 4, 7, 19, 25, 27, 13, 17, 18, 23, 3, 30, 15, 20, 11, 26, 5, 1, 16, 8, 24, 22, 10, 29, 21, 28, 2, 9, 12])

In [154]:
for i in range(0, ldamodel.num_topics):
>>>  print (ldamodel.print_topic(i))

0.053*"increase" + 0.042*"three" + 0.037*"average" + 0.029*"return" + 0.028*"change" + 0.028*"current" + 0.028*"four" + 0.025*"good" + 0.024*"year" + 0.024*"started"
0.234*"holding" + 0.133*"energy" + 0.063*"june" + 0.049*"#sabr#" + 0.041*"lot" + 0.034*"chinese" + 0.024*"oil_gas" + 0.024*"free" + 0.023*"medium" + 0.021*"get"
0.505*"target" + 0.267*"price" + 0.122*"start" + 0.025*"sector" + 0.020*"capital" + 0.012*"company" + 0.008*"funding" + 0.006*"technology" + 0.006*"round" + 0.005*"scale"
0.059*"share" + 0.045*"per" + 0.034*"closed" + 0.032*"stock" + 0.029*"offering" + 0.027*"public" + 0.027*"week" + 0.026*"initial" + 0.022*"range" + 0.021*"price"
0.087*"#n#" + 0.083*"investment" + 0.051*"resource" + 0.048*"ago" + 0.037*"provide" + 0.030*"selling" + 0.025*"banker" + 0.023*"may" + 0.021*"#txn#" + 0.019*"california"
0.045*"service" + 0.029*"solution" + 0.018*"data" + 0.018*"global" + 0.017*"cloud" + 0.017*"customer" + 0.016*"management" + 0.016*"industry" + 0.015*"network" + 0.014*"t

C:\Users\arjain\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
